In [1]:
import os
from dotenv import load_dotenv
from utils.ml_logging import get_logger

# Initialize logging
logger = get_logger()

# Load environment variables from .env file
load_dotenv()

# Define the target directory
target_directory = os.getcwd()  # change your directory here

# Check if the directory exists
if os.path.exists(target_directory):
    # Change the current working directory
    os.chdir(target_directory)
    print(f"Directory changed to {os.getcwd()}")
else:
    print(f"Directory {target_directory} does not exist.")

Directory changed to c:\Users\pablosal\Desktop\gbb-ai-hls-fsi-factory-insurance-autoauth


## **Approved Process**

When an underwriting application is **Approved**, it means that the applicant has successfully met all the required criteria outlined in the underwriting guidelines. The approval process ensures that the policy can be issued without further delays or additional information. Below is a breakdown of the key steps involved in the approval process:

1. **Validation of Criteria**:  
   - All underwriting criteria, such as applicant details, driver information, vehicle data, and coverage selections, have been reviewed and found to be in full compliance with the policy guidelines.

2. **Confirmation of Required Documentation**:  
   - All necessary documents, such as proof of prior insurance, vehicle inspection forms, and signed disclosures, have been provided and verified.

3. **Risk Assessment**:  
   - The applicant and associated risks (e.g., driving history, vehicle type, garaging location) fall within acceptable thresholds as defined by the underwriting policy.

4. **Policy Binding**:  
   - The policy is officially bound, and the effective date is confirmed. Payment plans or full payment details are finalized.

5. **Issuance of Policy Documents**:  
   - The approved policy documents, including declarations, coverage details, and terms, are generated and sent to the applicant and agent.

### **1. Gather User Inputs** 

In [2]:
from src.pipeline.promptEngineering.models import (
       AutoUnderwritingData,
       PolicyLevelInformation,
       ApplicantInformation,
       DriverDetail,
       VehicleDetail,
       CoverageSelections,
       DiscountInformation,
       SurchargeInformation
   )
from src.pipeline.promptEngineering.prompt_manager import PromptManager


policy_info = PolicyLevelInformation(
    policy_effective_date="2025-01-01",
    policy_term="6 Months",
    policy_type="Standard Auto",
    binding_date_time="2024-12-25 14:00:00",
    policy_payment_plan="Paid in Full",
    applicant_signature="ApplicantSignature123",
    agent_signature="AgentSignatureABC",
    um_uim_form="UM/UIM Accepted",
    named_driver_exclusion_form="None",
    misrep_disclosure="DisclosureAcknowledged"
)

applicant_info = ApplicantInformation(
    applicant_name="John Doe",
    applicant_dob="1980-05-15",
    applicant_id="JD123456",
    applicant_address="123 Main St, Columbus, OH 43215",
    applicant_mailing_address="PO Box 789, Columbus, OH 43216",
    applicant_phone="(614) 555-1212",
    applicant_email="john.doe@example.com",
    applicant_ssn="XXX-XX-1234",
    applicant_marital_status="Married",
    applicant_military_status="Active Duty In-State",
    applicant_homeowner_status="Yes"
)

driver_info = [
    DriverDetail(
        driver_name="John Doe",
        driver_dob="1980-05-15",
        driver_relationship="Self",
        driver_license_state="OH",
        driver_license_number="D123-4567-8901",
        driver_license_status="Valid",
        driver_sr22="No",
        driver_accident_violation_history="1 speeding ticket in last 12 months",
        driver_military_status="Active",
        driver_excluded="No"
    ),
    DriverDetail(
        driver_name="Jane Doe",
        driver_dob="1982-07-20",
        driver_relationship="Spouse",
        driver_license_state="OH",
        driver_license_number="D987-6543-2100",
        driver_license_status="Valid",
        driver_sr22="No",
        driver_accident_violation_history="No accidents/violations",
        driver_military_status="None",
        driver_excluded="No"
    )
]

vehicle_info = [
    VehicleDetail(
        vehicle_year="2022",
        vehicle_make="Toyota",
        vehicle_model="Camry",
        vehicle_vin="1ABC123DEF4567890",
        vehicle_ownership="Named Insured",
        vehicle_garaging_address="123 Main St, Columbus, OH 43215",
        vehicle_usage="Commute",
        vehicle_existing_damage="None",
        vehicle_age="1",
        vehicle_modifications="None",
        vehicle_salvage_title="No",
        vehicle_months_in_ohio="12",
        vehicle_class="Sedan",
        vehicle_prohibited_make="No"
    )
]

coverage_selections = CoverageSelections(
    coverage_liability_limits="25/50/25",
    coverage_um_uim="Accepted (25/50)",
    coverage_comp_deductible="$500",
    coverage_collision_deductible="$500",
    coverage_comp_collision_eligible="Yes",
    coverage_um_pd="$7,500",
    coverage_med_pay="$1,000",
    coverage_rental="$30/day ($900 max)",
    coverage_towing="$75 per disablement",
    coverage_custom_equip="None"
)

discounts = DiscountInformation(
    discount_prior_insurance="Yes (proof attached)",
    discount_homeowner_proof="Yes (mortgage statement)",
    discount_military_proof="Provided (Active ID)",
    discount_senior_defensive_driver="No",
    discount_auto_pay="Yes (EFT)",
    discount_paid_in_full="Yes",
    discount_advance_purchase="Bound 5 days before effective date"
)

surcharges = SurchargeInformation(
    surcharge_license_status="No surcharge",
    surcharge_at_fault_accidents="No at-fault accidents",
    surcharge_major_violations="None",
    surcharge_sr22="No",
    surcharge_policy_activity="No prior cancellations",
    surcharge_unacceptable_risk="No"
)



### **2. Gather policy**

In [3]:
from src.documentintelligence.document_intelligence_helper import AzureDocumentIntelligenceManager
text_extractor = AzureDocumentIntelligenceManager()
policy_raw_text_markdown = text_extractor.analyze_document(document_input="https://storageaeastusfactory.blob.core.windows.net/insurance-policies/OH-MGA-Personal-Auto-Program-Rules-Manual.pdf", 
                                model_type="prebuilt-layout")

2025-04-01 13:06:17,698 - micro - MainProcess - INFO     Container 'pre-auth-policies' already exists. (blob_helper.py:_create_container_if_not_exists:89)
2025-04-01 13:06:17,699 - micro - MainProcess - INFO     Blob URL detected. Extracting content. (document_intelligence_helper.py:analyze_document:78)
2025-04-01 13:06:18,042 - micro - MainProcess - INFO     Downloaded blob 'OH-MGA-Personal-Auto-Program-Rules-Manual.pdf' as bytes. (blob_helper.py:download_blob_to_bytes:311)


### **3. Build Prompts**

In [4]:
auto_data = AutoUnderwritingData(
    policy_level=policy_info,
    applicant_info=applicant_info,
    driver_info=driver_info,
    vehicle_info=vehicle_info,
    coverage_selections=coverage_selections,
    discounts=discounts,
    surcharges=surcharges,
    policy_text=policy_raw_text_markdown.content
)

prompt_manager = PromptManager(template_dir="templates")

try:
    underwriting_user_prompt = prompt_manager.create_prompt_auto_user_underwriting(auto_data)
except Exception as e:
    print(f"Error generating prompt: {e}")

Templates in c:\Users\pablosal\Desktop\gbb-ai-hls-fsi-factory-insurance-autoauth\src\pipeline\promptEngineering\templates: ['underwriting_o1_system_prompt.jinja', 'underwriting_o1_user_prompt.jinja']


### **4. Run Auto Underwriting**

In [6]:
from src.aoai.aoai_helper import AzureOpenAIManager
llm = AzureOpenAIManager(api_version=os.getenv("AZURE_OPENAI_API_VERSION_01"),
                        chat_model_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_01"))

In [7]:
underwriting_decision = await llm.generate_chat_response_o1(query=underwriting_user_prompt)

2025-04-01 13:06:48,187 - micro - MainProcess - INFO     Function generate_chat_response_o1 started at 2025-04-01 13:06:48 (aoai_helper.py:generate_chat_response_o1:274)
2025-04-01 13:06:48,189 - micro - MainProcess - INFO     Sending request to Azure OpenAI at 2025-04-01 13:06:48 (aoai_helper.py:generate_chat_response_o1:282)
2025-04-01 13:07:12,111 - micro - MainProcess - INFO     Model_used: o1-2024-12-17 (aoai_helper.py:generate_chat_response_o1:306)
2025-04-01 13:07:12,113 - micro - MainProcess - INFO     Function generate_chat_response_o1 finished at 2025-04-01 13:07:12 (Duration: 23.93 seconds) (aoai_helper.py:generate_chat_response_o1:315)


In [8]:
print(underwriting_decision['response'])

Underwriting Decision  
Approved  

Rationale  

1. Summary of Findings  
   The applicant, John Doe, and his spouse, Jane Doe, meet all the stated underwriting criteria. They both hold valid Ohio driver’s licenses, the single speeding ticket for John is within acceptable limits, the 2022 Toyota Camry is neither an older model nor an unacceptable make/model, and all required documentation for discounts (prior insurance, homeownership, military status, etc.) is provided. There are no apparent violations of the Unacceptable Risk guidelines, and the policy requirements for binding and documentation appear fully satisfied.

2. Detailed Analysis  

   Policy Criteria Assessment  
   a) Criterion: “All information must be submitted using GAINSCOconnect.com; application must be complete, signed, with necessary documentation.”  
      • Assessment: Fully Met  
      • Evidence: Application is signed by both applicant (ApplicantSignature123) and agent (AgentSignatureABC). UM/UIM acceptance is d

## **Denied Process**

When an underwriting application is **Denied**, it means that the applicant has failed to meet one or more of the required criteria outlined in the underwriting guidelines. The denial process ensures that the decision is communicated clearly and that the reasons for denial are well-documented. Below is a breakdown of the key steps involved in the denial process:

1. **Identification of Non-Compliance**:  
   - Specific underwriting criteria that were not met are identified, such as unacceptable driving history, prohibited vehicle types, missing documentation, or other policy violations.

2. **Documentation of Reasons for Denial**:  
   - A detailed explanation of the reasons for denial is prepared, referencing the specific underwriting guidelines or criteria that were not satisfied.  
   - Examples include:  
     - **Driver History**: Excessive violations or at-fault accidents.  
     - **Vehicle Issues**: Prohibited make/model or salvage/branded title.  
     - **Missing Information**: Required documents such as proof of prior insurance or signed disclosures were not provided.

3. **Risk Assessment Confirmation**:  
   - The applicant's risk profile is reviewed to ensure that the denial aligns with the underwriting policy and is not based on incomplete or incorrect data.

4. **Communication of Denial**:  
   - The applicant and agent are notified of the denial decision in a clear and professional manner.  
   - The communication includes:  
     - A summary of the reasons for denial.  
     - Any potential steps the applicant can take to address the issues (if applicable).  
     - Contact information for further inquiries.

5. **Record Keeping**:  
   - The denial decision, along with supporting documentation and rationale, is recorded in the underwriting system for future reference and compliance purposes.

By following this process, the insurer ensures that denial decisions are fair, transparent, and compliant with underwriting standards while maintaining professionalism in communication.

In [25]:
from src.pipeline.promptEngineering.models import (
       AutoUnderwritingData,
       PolicyLevelInformation,
       ApplicantInformation,
       DriverDetail,
       VehicleDetail,
       CoverageSelections,
       DiscountInformation,
       SurchargeInformation
   )
from src.pipeline.promptEngineering.prompt_manager import PromptManager


policy_info = PolicyLevelInformation(
    policy_effective_date="2025-01-01",
    policy_term="6 Months",
    policy_type="Standard Auto",
    binding_date_time="2024-12-25 14:00:00",
    policy_payment_plan="Paid in Full",
    applicant_signature="ApplicantSignature123",
    agent_signature="AgentSignatureABC",
    um_uim_form="UM/UIM Accepted",
    named_driver_exclusion_form="None",
    misrep_disclosure="DisclosureAcknowledged"
)

applicant_info = ApplicantInformation(
    applicant_name="John Doe",
    applicant_dob="1980-05-15",
    applicant_id="JD123456",
    applicant_address="123 Main St, Columbus, OH 43215",
    applicant_mailing_address="PO Box 789, Columbus, OH 43216",
    applicant_phone="(614) 555-1212",
    applicant_email="john.doe@example.com",
    applicant_ssn="XXX-XX-1234",
    applicant_marital_status="Married",
    applicant_military_status="Active Duty In-State",
    applicant_homeowner_status="Yes"
)

driver_info = [
    DriverDetail(
        driver_name="John Doe",
        driver_dob="1980-05-15",
        driver_relationship="Self",
        driver_license_state="OH",
        driver_license_number="D123-4567-8901",
        driver_license_status="Valid",
        driver_sr22="No",
        driver_accident_violation_history="1 speeding ticket in last 12 months",
        driver_military_status="Active",
        driver_excluded="No"
    ),
    DriverDetail(
        driver_name="Jane Doe",
        driver_dob="1982-07-20",
        driver_relationship="Spouse",
        driver_license_state="OH",
        driver_license_number="D987-6543-2100",
        driver_license_status="Valid",
        driver_sr22="No",
        driver_accident_violation_history="No accidents/violations",
        driver_military_status="None",
        driver_excluded="No"
    )
]

vehicle_info = [
    VehicleDetail(
        vehicle_year="2022",
        vehicle_make="Toyota",
        vehicle_model="Camry",
        vehicle_vin="1ABC123DEF4567890",
        vehicle_ownership="Named Insured",
        vehicle_garaging_address="123 Main St, Columbus, OH 43215",
        vehicle_usage="Commute",
        vehicle_existing_damage="None",
        vehicle_age="1",
        vehicle_modifications="None",
        vehicle_salvage_title="No",
        vehicle_months_in_ohio="8",  # Not garaged in Ohio 10+ months
        vehicle_class="Sedan",
        vehicle_prohibited_make="No"
    )
]

coverage_selections = CoverageSelections(
    coverage_liability_limits="25/50/25",
    coverage_um_uim="Accepted (25/50)",
    coverage_comp_deductible="$500",
    coverage_collision_deductible="$500",
    coverage_comp_collision_eligible="Yes",
    coverage_um_pd="$7,500",
    coverage_med_pay="$1,000",
    coverage_rental="$30/day ($900 max)",
    coverage_towing="$75 per disablement",
    coverage_custom_equip="$2,000"   # Additional equipment coverage (wheels/stereo)
)

discounts = DiscountInformation(
    discount_prior_insurance="Yes (proof attached)",
    discount_homeowner_proof="Yes (mortgage statement)",
    discount_military_proof="Provided (Active ID)",
    discount_senior_defensive_driver="No",
    discount_auto_pay="Yes (EFT)",
    discount_paid_in_full="Yes",
    discount_advance_purchase="Bound 5 days before effective date"
)

surcharges = SurchargeInformation(
    surcharge_license_status="No surcharge",
    surcharge_at_fault_accidents="No at-fault accidents",
    surcharge_major_violations="None",
    surcharge_sr22="No",
    surcharge_policy_activity="No prior cancellations",
    surcharge_unacceptable_risk="No"
)



In [26]:
auto_data = AutoUnderwritingData(
    policy_level=policy_info,
    applicant_info=applicant_info,
    driver_info=driver_info,
    vehicle_info=vehicle_info,
    coverage_selections=coverage_selections,
    discounts=discounts,
    surcharges=surcharges,
    policy_text=policy_raw_text_markdown.content
)

prompt_manager = PromptManager(template_dir="templates")

try:
    underwriting_user_prompt = prompt_manager.create_prompt_auto_user_underwriting(auto_data)
except Exception as e:
    print(f"Error generating prompt: {e}")

Templates in c:\Users\pablosal\Desktop\gbb-ai-hls-fsi-factory-insurance-autoauth\src\pipeline\promptEngineering\templates: ['underwriting_o1_system_prompt.jinja', 'underwriting_o1_user_prompt.jinja']


In [27]:
from src.aoai.aoai_helper import AzureOpenAIManager
llm = AzureOpenAIManager(api_version=os.getenv("AZURE_OPENAI_API_VERSION_01"),
                        chat_model_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_01"))

In [28]:
underwriting_decision = await llm.generate_chat_response_o1(query=underwriting_user_prompt)

2025-04-01 14:18:12,397 - micro - MainProcess - INFO     Function generate_chat_response_o1 started at 2025-04-01 14:18:12 (aoai_helper.py:generate_chat_response_o1:274)
2025-04-01 14:18:12,400 - micro - MainProcess - INFO     Sending request to Azure OpenAI at 2025-04-01 14:18:12 (aoai_helper.py:generate_chat_response_o1:282)
2025-04-01 14:18:28,823 - micro - MainProcess - INFO     Model_used: o1-2024-12-17 (aoai_helper.py:generate_chat_response_o1:306)
2025-04-01 14:18:28,826 - micro - MainProcess - INFO     Function generate_chat_response_o1 finished at 2025-04-01 14:18:28 (Duration: 16.43 seconds) (aoai_helper.py:generate_chat_response_o1:315)


In [29]:
print(underwriting_decision['response'])

Underwriting Decision  
Denied  

Rationale  

1. Summary of Findings  
   Although most underwriting requirements are satisfied, the vehicle fails the mandatory “10 months in Ohio” garaging requirement. The applicant states the Toyota Camry is garaged in Ohio only 8 months per year, which explicitly appears as an unacceptable risk in the underwriting guidelines.

2. Detailed Analysis  

   Policy Criteria Assessment  

   • Criterion: “Vehicles garaged in Ohio less than 10 months per year are unacceptable.”  
     – Assessment: Not Met  
     – Evidence: The application specifies the vehicle is garaged in Ohio for only 8 months.  
     – Policy Reference: “Vehicles,” Under “Unacceptable Vehicles” (p. 8), which states “Vehicles garaged in Ohio less than 10 months per year” are unacceptable.

   All other criteria—such as valid driver licensing, prior insurance, required signatures, and discount documentation—appear to be met. However, the 8-month garaging status directly conflicts with